<a href="https://colab.research.google.com/github/r1datascience/FoCS/blob/main/Progetto_FoCS_21_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Foundations of Computer Science** -  a.a. 2021-22   (Rebecca Picarelli n°mat. 834286)









Come prima cosa, si procede con l'importazione delle librerie Python uyili per il progetto

In [1]:
import pandas as pd
import numpy as np

# 1. Normalize the location field of Users dataset, splitting into city, region, country.



In [58]:
# Caricamento del dataset Users.csv
users = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz', compression='gzip', encoding='latin-1', delimiter=';')
users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


Si possono controllare eventuali righe duplicate nel dataset

In [67]:
users.duplicated().any()

False

Ci sono diverse tipologie di errori all'interno della variabile "Location". Ad esempio, si possono osservare problemi di codifica dei nomi, mancata corrispondenza tra nome della città/regione e stato, presenza di ipotetici indirizzi al posto del semplice nome secondo i 3 parametri.
Quindi, con il seguente codice si cercherà di contare ad alcune di queste problematiche riscontrate. Inoltre, non si opterà per rimuoverle, in quanto la loro presenza non comporta nessun disturbo viste le richieste successive da eseguire sul dataset "user".

In [60]:
#In valore assoluto si avrebbero almeno 9744 righe problematiche (senza considerare errori geografici)
len(users[users['Location'].str.contains(r"[1234567890ÃÂîûúöð§Ô½©£$€¶¤¼@¹¾‹«»›%±]")])

9744

In [59]:
#Mentre a livello di percentuale, ciò equivale allo 0,03% delle righe del dataset "users"
len(users[users['Location'].str.contains(r"[1234567890ÃÂîûúöð§Ô½©£$€¶¤¼@¹¾‹«»›%±]")])/len(users) 

0.034942515545546476

Vedere il numero di location diverse raggruppando secondo la variabile "Location"

In [ ]:
users.groupby(['Location'])['User-ID'].count().reset_index() #Apparentemente ci sono 57339 righe diverse secondo la variabile "Location"

,Location,User-ID
0,"&#19978;&#28023;, shanghai, china",1
1,&#19978;&#28023;/&#21488;&#24030;/&#36335;&#26...,1
2,&#20013;&#22269;&#24191;&#19996;&#30465;&#2426...,1
3,&#20013;&#22269;&#27993;&#27743;&#30465;&#2770...,1
4,"&#20013;&#23665;&#24066;, puerto rico, hong kong",1
...,...,...
57334,"Ã½stanbul, Ã½stanbul, turkey",5
57335,"Ã½stanbul, Ã½stanbul/anadolu, turkey",1
57336,"Ã½zmir, connecticut, turkey",1
57337,"Ã½zmir, n/a, turkey",4


Provare a contare i missing, anche se ovviamente ce ne dovrebbero essere di più se si considerano i dati mancanti/mal inseriri in "Location" che non vengono rintracciati

In [ ]:
#Valori in percentuale
missing=users.isnull().sum() * 100 / len(users)
missing

User-ID      0.000000
Location     0.000000
Age         39.719857
dtype: float64

Sistemare il formato della variabile "Age" (da float a intero),per poi vederene la distribuzione
Si può osservare la presenza di utenti particolarmente anziani. Probabilmente ciò può essere dovuto a qualche errore nell'inserimento dei dati (?)

In [ ]:
users['Age'].astype('Int64')
users['Age'].describe()

count    168096.000000
mean         34.751434
std          14.428097
min           0.000000
25%          24.000000
50%          32.000000
75%          44.000000
max         244.000000
Name: Age, dtype: float64

In [61]:
users.groupby('Age').count()

,User-ID,Location
Age,,
0.0,416,416
1.0,288,288
2.0,105,105
3.0,45,45
4.0,28,28
...,...,...
230.0,1,1
231.0,1,1
237.0,1,1


Splittare la variabile "Location" in tre nuove colonne "City", "Region","Country", eliminando la colonna originale

In [3]:
users[['City', 'Region', 'Country']] = users['Location'].str.split(',',n=2, expand=True)
users.drop('Location', 1, inplace=True)
users

,User-ID,Age,City,Region,Country
0,1,NaN,nyc,new york,usa
1,2,18.0,stockton,california,usa
2,3,NaN,moscow,yukon territory,russia
3,4,17.0,porto,v.n.gaia,portugal
4,5,NaN,farnborough,hants,united kingdom
...,...,...,...,...,...
278853,278854,NaN,portland,oregon,usa
278854,278855,50.0,tacoma,washington,united kingdom
278855,278856,NaN,brampton,ontario,canada
278856,278857,NaN,knoxville,tennessee,usa


# 2. For each book in the Books dataset, compute its average rating.

Caricamento del dataset Books.csv

In [65]:
books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz', compression='gzip', encoding='latin-1', delimiter=';')
books

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271354,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271355,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271356,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271357,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


Controllare eventuali righe duplicate

In [69]:
books.duplicated().any()


False

Caricamento del dataset Book-Ratings.csv

In [70]:
book_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz', compression='gzip', encoding='latin-1', delimiter=';')
book_ratings  #1149780 rows × 3 columns


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


Controllare eventuali righe duplicate 

In [71]:
book_ratings.duplicated().any() #Non sono presenti righe duplicate

False

Effettuare un raggruppamento per codice ISBN in book_ratings per poi calcolare la medai dei valori di Book-Rating, che poi sarà da unire al dataset books, rinominato "book_ratings_average"

In [6]:
books_ratings_average = pd.merge(books[['ISBN', 'Book-Title', 'Book-Author', 'Publisher']], book_ratings.groupby('ISBN')['Book-Rating'].mean(), on='ISBN', how='left')
books_ratings_average

,ISBN,Book-Title,Book-Author,Publisher,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,Oxford University Press,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,4.928571
2,0060973129,Decision in Normandy,Carlo D'Este,HarperPerennial,5.000000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Farrar Straus Giroux,4.272727
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton &amp; Company,0.000000
...,...,...,...,...,...
271354,0440400988,There's a Bat in Bunk Five,Paula Danziger,Random House Childrens Pub (Mm),7.000000
271355,0525447644,From One to One Hundred,Teri Sloat,Dutton Books,4.000000
271356,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,HarperSanFrancisco,0.000000
271357,0192126040,Republic (World's Classics),Plato,Oxford University Press,0.000000


In [7]:
books_ratings_average.rename({'Book-Rating': 'Book-Rating_average'}, axis=1, inplace=True)
books_ratings_average['Book-Rating_average']=books_ratings_average['Book-Rating_average'].round(1)
books_ratings_average

,ISBN,Book-Title,Book-Author,Publisher,Book-Rating_average
0,0195153448,Classical Mythology,Mark P. O. Morford,Oxford University Press,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,4.9
2,0060973129,Decision in Normandy,Carlo D'Este,HarperPerennial,5.0
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Farrar Straus Giroux,4.3
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton &amp; Company,0.0
...,...,...,...,...,...
271354,0440400988,There's a Bat in Bunk Five,Paula Danziger,Random House Childrens Pub (Mm),7.0
271355,0525447644,From One to One Hundred,Teri Sloat,Dutton Books,4.0
271356,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,HarperSanFrancisco,0.0
271357,0192126040,Republic (World's Classics),Plato,Oxford University Press,0.0


# 3. For each book in the GoodBooks dataset, compute its average rating.

Caricamento del dataset goodbooks.csv

In [8]:
goodbooks = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz', compression='gzip', encoding='latin-1', delimiter=',')
goodbooks #99 rows

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,5297,5297,1858012,2303,375751513,9.780376e+12,"Oscar Wilde, Jeffrey Eugenides",1891.0,The Picture of Dorian Gray,The Picture of Dorian Gray,eng,4.06,590014,674551,19647,9825,30735,129420,246892,257679,https://images.gr-assets.com/books/1424596966m...,https://images.gr-assets.com/books/1424596966s...
95,96,13536860,13536860,18034963,133,345803507,9.780346e+12,E.L. James,2012.0,Fifty Shades Freed,"Fifty Shades Freed (Fifty Shades, #3)",eng,3.88,387290,610116,25287,39079,55691,109963,140304,265079,https://images.gr-assets.com/books/1336418837m...,https://images.gr-assets.com/books/1336418837s...
96,97,17245,17245,3165724,2207,393970124,9.780394e+12,"Bram Stoker, Nina Auerbach, David J. Skal",1897.0,Dracula,Dracula,eng,3.98,618973,689739,17263,11756,36719,151396,246043,243825,https://images.gr-assets.com/books/1387151694m...,https://images.gr-assets.com/books/1387151694s...
97,98,5060378,5060378,6976108,209,307269981,9.780307e+12,"Stieg Larsson, Reg Keeland",2006.0,Flickan som lekte med elden,"The Girl Who Played with Fire (Millennium, #2)",eng,4.22,563994,649253,31423,4623,15781,91021,258236,279592,https://images.gr-assets.com/books/1351778881m...,https://images.gr-assets.com/books/1351778881s...


Caricamento del dataset goodbooks-ratings.csv

In [9]:
gb_ratings= pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz', compression='gzip', encoding='latin-1', delimiter=',')
gb_ratings #99 rows x 3 cols

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
94,4,184,4
95,4,495,4
96,4,350,3
97,4,492,4


In [10]:
gb_ratings_average = gb_ratings.groupby(['book_id'], as_index=False).mean()[['book_id','rating']]
gb_ratings_average

,book_id,rating
0,2,5.0
1,5,4.0
2,8,4.0
3,13,4.0
4,14,5.0
...,...,...
91,6195,4.0
92,6351,4.0
93,8519,5.0
94,9114,5.0


Effettuare una merge tra goodbooks e gb_ratings_average per vedere anche gli altri dati abbinati alla media appena calcolata

In [11]:
gb_ratings_final = pd.merge(goodbooks, gb_ratings_average, on='book_id', how='left')
gb_ratings_final.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,rating
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,NaN
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,5.0
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,NaN
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,NaN
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,4.0


# 4.Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset merged books. The books that have not been merged together will not appear in merged books


individuare libri 
(tramite titolo, autore e casa editrice) che nel database appaiono
 più volte (con ISBN diversi), perchè ad esempio ripubblicati a distanza di anni.



Creare un dataset con la chiave titolo-autore-publisher e con il numero di righe (count) raggruppate in ogni gruppo (si andranno ad eliminare i count=1)

In [12]:
merged_books = books.groupby(['Book-Title','Book-Author','Publisher'])['Book-Title'].count().to_frame(name = 'count').reset_index()
merged_books


,Book-Title,Book-Author,Publisher,count
0,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,Hyperion Books for Children,1
1,Always Have Popsicles,Rebecca Harvin,Rebecca L. Harvin,1
2,Apple Magic (The Collector's series),Martina Boudreau,Amer Cooking Guild,1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",Nancy N. Rue,Zonderkidz,1
4,Beyond IBM: Leadership Marketing and Finance ...,Lou Mobley,"Teleonet, Incorporated",1
...,...,...,...,...
266122,Ã?Ã?lpiraten.,Janwillem van de Wetering,Rowohlt Tb.,1
266123,Ã?Ã?rger mit Produkt X. Roman.,Joan Aiken,Diogenes Verlag,1
266124,Ã?Ã?sterlich leben.,Anselm GrÃ?ÃÂ¼n,"Herder, Freiburg",1
266125,Ã?Ã?stlich der Berge.,David Guterson,btb,1


Distribuzione dei valori in 'count'

In [ ]:
merged_books['count'].value_counts()

1     261402
2       4336
3        313
4         56
5         11
6          5
7          2
12         1
8          1
Name: count, dtype: int64

Rimuovere i casi che non rappresentano raggruppamenti

In [13]:
merged_books=merged_books.loc[merged_books['count'] != 1]
merged_books

,Book-Title,Book-Author,Publisher,count
41,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2
59,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2
255,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2
465,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3
713,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,2
...,...,...,...,...
265525,Zia,Scott O'Dell,Laurel-Leaf Books,2
265526,Zia,Scott O'Dell,Yearling Books,2
265562,Zimmermann Telegram,Barbara Tuchman,Ballantine Books,2
265625,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,4


#5. For each book in merged books compute its average rating.

The average is computed considering all books in books that have been merged.((RAGGRUPPARE PER la tupla (title, author e publisher)))

In [14]:
merged_books_average = merged_books.merge(books_ratings_average, on=['Book-Title', 'Book-Author', 'Publisher'], how='left')
merged_books_average

,Book-Title,Book-Author,Publisher,count,ISBN,Book-Rating_average
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2,1565920465,0.0
1,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2,1565920317,6.0
2,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792276833,0.0
3,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792277295,3.4
4,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2,1566196256,7.2
...,...,...,...,...,...,...
9949,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,4,1569317674,10.0
9950,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,4,1569317666,10.0
9951,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,4,1569317658,10.0
9952,Ã?Ã?berleben in der WÃ?ÃÂ¼ste Danakil.,RÃ?ÃÂ¼diger Nehberg,Piper,2,3492218091,5.0


In [15]:
merged_books_avg_rating=merged_books_average.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False)['Book-Rating_average'].mean()
merged_books_avg_rating

,Book-Title,Book-Author,Publisher,Book-Rating_average
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,3.00
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,1.70
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,6.95
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3.10
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,0.00
...,...,...,...,...
4720,Zia,Scott O'Dell,Laurel-Leaf Books,2.55
4721,Zia,Scott O'Dell,Yearling Books,1.00
4722,Zimmermann Telegram,Barbara Tuchman,Ballantine Books,2.00
4723,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.00


#6. For each book in merged books compute the minimum and maximum of the average ratings over all corresponding books in the books dataset.
Hence for each book in merged books we will have exactly two values (a minimum and a maximum) ((In books non c'è il rating.
Per ogni libro, deve calcolare il rating medio, e di quest'ultimo estrarre minimo e massimo.))

In [16]:
merged_books_average['min_rating'] = merged_books_average.groupby(['Book-Title','Book-Author'], as_index=False)['Book-Rating_average'].transform('min')

merged_books_average

,Book-Title,Book-Author,Publisher,count,ISBN,Book-Rating_average,min_rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2,1565920465,0.0,0.0
1,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2,1565920317,6.0,0.0
2,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792276833,0.0,0.0
3,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792277295,3.4,0.0
4,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2,1566196256,7.2,6.7
...,...,...,...,...,...,...,...
9949,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,4,1569317674,10.0,10.0
9950,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,4,1569317666,10.0,10.0
9951,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,4,1569317658,10.0,10.0
9952,Ã?Ã?berleben in der WÃ?ÃÂ¼ste Danakil.,RÃ?ÃÂ¼diger Nehberg,Piper,2,3492218091,5.0,2.7


In [17]:
merged_books_average['max_rating'] = merged_books_average.groupby(['Book-Title','Book-Author'], as_index=False)['Book-Rating_average'].transform('max')
merged_books_average

,Book-Title,Book-Author,Publisher,count,ISBN,Book-Rating_average,min_rating,max_rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2,1565920465,0.0,0.0,6.0
1,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2,1565920317,6.0,0.0,6.0
2,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792276833,0.0,0.0,3.4
3,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2,0792277295,3.4,0.0,3.4
4,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2,1566196256,7.2,6.7,7.2
...,...,...,...,...,...,...,...,...
9949,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,4,1569317674,10.0,10.0,10.0
9950,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,4,1569317666,10.0,10.0,10.0
9951,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,4,1569317658,10.0,10.0,10.0
9952,Ã?Ã?berleben in der WÃ?ÃÂ¼ste Danakil.,RÃ?ÃÂ¼diger Nehberg,Piper,2,3492218091,5.0,2.7,5.0


In [18]:
#rimuovere i duplicati
merged_books_average = merged_books_average[merged_books_average.columns.drop(['Book-Rating_average', 'ISBN','count'])]
merged_books_average = merged_books_average.drop_duplicates()

In [19]:
merged_books_average

,Book-Title,Book-Author,Publisher,min_rating,max_rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,0.0,6.0
2,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,0.0,3.4
4,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,6.7,7.2
6,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,0.0,5.0
9,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,0.0,0.0
...,...,...,...,...,...
9942,Zia,Scott O'Dell,Laurel-Leaf Books,0.0,3.4
9944,Zia,Scott O'Dell,Yearling Books,0.0,3.4
9946,Zimmermann Telegram,Barbara Tuchman,Ballantine Books,0.0,4.0
9948,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.0,10.0


# 7. For each book in goodbooks, compute the list of its authors. Assuming that the number of reviews with a text (column work_text_reviews_count) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the shared number of reviews with a text.


per ogni autore: tot di reviews witha  text(=shared n of reviews with a text)

In [20]:
#Contare quanti autori ci sono per ogni libro
goodbooks['authors_count'] = goodbooks['authors'].str.split(',').str.len()
goodbooks

#gb_list = good_books.join(good_books.pop('authors').str.split(',', expand=True))
#gb_list

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,authors_count
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,1
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,2
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,1
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,1
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,5297,5297,1858012,2303,375751513,9.780376e+12,"Oscar Wilde, Jeffrey Eugenides",1891.0,The Picture of Dorian Gray,The Picture of Dorian Gray,eng,4.06,590014,674551,19647,9825,30735,129420,246892,257679,https://images.gr-assets.com/books/1424596966m...,https://images.gr-assets.com/books/1424596966s...,2
95,96,13536860,13536860,18034963,133,345803507,9.780346e+12,E.L. James,2012.0,Fifty Shades Freed,"Fifty Shades Freed (Fifty Shades, #3)",eng,3.88,387290,610116,25287,39079,55691,109963,140304,265079,https://images.gr-assets.com/books/1336418837m...,https://images.gr-assets.com/books/1336418837s...,1
96,97,17245,17245,3165724,2207,393970124,9.780394e+12,"Bram Stoker, Nina Auerbach, David J. Skal",1897.0,Dracula,Dracula,eng,3.98,618973,689739,17263,11756,36719,151396,246043,243825,https://images.gr-assets.com/books/1387151694m...,https://images.gr-assets.com/books/1387151694s...,3
97,98,5060378,5060378,6976108,209,307269981,9.780307e+12,"Stieg Larsson, Reg Keeland",2006.0,Flickan som lekte med elden,"The Girl Who Played with Fire (Millennium, #2)",eng,4.22,563994,649253,31423,4623,15781,91021,258236,279592,https://images.gr-assets.com/books/1351778881m...,https://images.gr-assets.com/books/1351778881s...,2


In [21]:
goodbooks.authors = goodbooks.authors.str.split(', ')
gb_authors =goodbooks.explode('authors')#.set_index('authors')
gb_authors

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,authors_count
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,1
1,2,3,3,4640799,491,439554934,9.780440e+12,J.K. Rowling,1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,2
1,2,3,3,4640799,491,439554934,9.780440e+12,Mary GrandPrÃ©,1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,2
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,1
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,97,17245,17245,3165724,2207,393970124,9.780394e+12,Nina Auerbach,1897.0,Dracula,Dracula,eng,3.98,618973,689739,17263,11756,36719,151396,246043,243825,https://images.gr-assets.com/books/1387151694m...,https://images.gr-assets.com/books/1387151694s...,3
96,97,17245,17245,3165724,2207,393970124,9.780394e+12,David J. Skal,1897.0,Dracula,Dracula,eng,3.98,618973,689739,17263,11756,36719,151396,246043,243825,https://images.gr-assets.com/books/1387151694m...,https://images.gr-assets.com/books/1387151694s...,3
97,98,5060378,5060378,6976108,209,307269981,9.780307e+12,Stieg Larsson,2006.0,Flickan som lekte med elden,"The Girl Who Played with Fire (Millennium, #2)",eng,4.22,563994,649253,31423,4623,15781,91021,258236,279592,https://images.gr-assets.com/books/1351778881m...,https://images.gr-assets.com/books/1351778881s...,2
97,98,5060378,5060378,6976108,209,307269981,9.780307e+12,Reg Keeland,2006.0,Flickan som lekte med elden,"The Girl Who Played with Fire (Millennium, #2)",eng,4.22,563994,649253,31423,4623,15781,91021,258236,279592,https://images.gr-assets.com/books/1351778881m...,https://images.gr-assets.com/books/1351778881s...,2


In [22]:
#find for each authors the total number of reviews with a text. We will call this quantity the shared number of reviews with a text
gb_authors['shared_n_text_reviews'] = gb_authors['work_text_reviews_count']/gb_authors['authors_count'] 
gb_authors

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,authors_count,shared_n_text_reviews
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,1,155254.000000
1,2,3,3,4640799,491,439554934,9.780440e+12,J.K. Rowling,1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,2,37933.500000
1,2,3,3,4640799,491,439554934,9.780440e+12,Mary GrandPrÃ©,1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,2,37933.500000
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,1,95009.000000
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,1,72586.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,97,17245,17245,3165724,2207,393970124,9.780394e+12,Nina Auerbach,1897.0,Dracula,Dracula,eng,3.98,618973,689739,17263,11756,36719,151396,246043,243825,https://images.gr-assets.com/books/1387151694m...,https://images.gr-assets.com/books/1387151694s...,3,5754.333333
96,97,17245,17245,3165724,2207,393970124,9.780394e+12,David J. Skal,1897.0,Dracula,Dracula,eng,3.98,618973,689739,17263,11756,36719,151396,246043,243825,https://images.gr-assets.com/books/1387151694m...,https://images.gr-assets.com/books/1387151694s...,3,5754.333333
97,98,5060378,5060378,6976108,209,307269981,9.780307e+12,Stieg Larsson,2006.0,Flickan som lekte med elden,"The Girl Who Played with Fire (Millennium, #2)",eng,4.22,563994,649253,31423,4623,15781,91021,258236,279592,https://images.gr-assets.com/books/1351778881m...,https://images.gr-assets.com/books/1351778881s...,2,15711.500000
97,98,5060378,5060378,6976108,209,307269981,9.780307e+12,Reg Keeland,2006.0,Flickan som lekte med elden,"The Girl Who Played with Fire (Millennium, #2)",eng,4.22,563994,649253,31423,4623,15781,91021,258236,279592,https://images.gr-assets.com/books/1351778881m...,https://images.gr-assets.com/books/1351778881s...,2,15711.500000


In [ ]:
gb_authors['shared_n_text_reviews'].dtype

dtype('float64')

In [23]:
gb_authors['shared_n_text_reviews']=gb_authors['shared_n_text_reviews'].astype(float).round(1)
gb_authors

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,authors_count,shared_n_text_reviews
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,1,155254.0
1,2,3,3,4640799,491,439554934,9.780440e+12,J.K. Rowling,1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,2,37933.5
1,2,3,3,4640799,491,439554934,9.780440e+12,Mary GrandPrÃ©,1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,2,37933.5
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,1,95009.0
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,1,72586.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,97,17245,17245,3165724,2207,393970124,9.780394e+12,Nina Auerbach,1897.0,Dracula,Dracula,eng,3.98,618973,689739,17263,11756,36719,151396,246043,243825,https://images.gr-assets.com/books/1387151694m...,https://images.gr-assets.com/books/1387151694s...,3,5754.3
96,97,17245,17245,3165724,2207,393970124,9.780394e+12,David J. Skal,1897.0,Dracula,Dracula,eng,3.98,618973,689739,17263,11756,36719,151396,246043,243825,https://images.gr-assets.com/books/1387151694m...,https://images.gr-assets.com/books/1387151694s...,3,5754.3
97,98,5060378,5060378,6976108,209,307269981,9.780307e+12,Stieg Larsson,2006.0,Flickan som lekte med elden,"The Girl Who Played with Fire (Millennium, #2)",eng,4.22,563994,649253,31423,4623,15781,91021,258236,279592,https://images.gr-assets.com/books/1351778881m...,https://images.gr-assets.com/books/1351778881s...,2,15711.5
97,98,5060378,5060378,6976108,209,307269981,9.780307e+12,Reg Keeland,2006.0,Flickan som lekte med elden,"The Girl Who Played with Fire (Millennium, #2)",eng,4.22,563994,649253,31423,4623,15781,91021,258236,279592,https://images.gr-assets.com/books/1351778881m...,https://images.gr-assets.com/books/1351778881s...,2,15711.5


In [25]:
author_reviews_tot = gb_authors.groupby('authors')['shared_n_text_reviews'].sum()
author_reviews_tot

authors
Alan R. Clarke                27890.5
Aldous Huxley                 20095.0
Alice Sebold                  36642.0
Anne Frank                     6941.7
Antoine de Saint-ExupÃ©ry      6134.2
                               ...   
Veronica Roth                156896.0
William Golding               26886.0
William Goldman               15630.0
William Shakespeare            7389.0
Yann Martel                   42962.0
Name: shared_n_text_reviews, Length: 109, dtype: float64

# 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.
 (Per ogni anno di pubblicazione, bisogna considerare tutti i libri pubblicati quell'anno, tutte le recensioni relative a tali libri, e determinare quale autore ha ricevuto più recensioni con testo,quindi per ogni anno, valore max delle reviews di ogni autore)


In [26]:
gb_max_reviews=gb_authors.groupby(['original_publication_year','authors'], as_index=False)[ 'shared_n_text_reviews'].max()
gb_max_reviews

,original_publication_year,authors,shared_n_text_reviews
0,-720.0,Bernard Knox,1620.2
1,-720.0,E.V. Rieu,1620.2
2,-720.0,FrÃ©dÃ©ric Mugler,1620.2
3,-720.0,Homer,1620.2
4,-720.0,Robert Fagles,1620.2
...,...,...,...
133,2012.0,E.L. James,25287.0
134,2012.0,Gillian Flynn,121614.0
135,2012.0,John Green,140739.0
136,2012.0,Veronica Roth,55873.0


Ricavare il nome degli autori che risultano avere il numero massimo per ogni anno

In [27]:
author_max_year = gb_max_reviews[gb_max_reviews.groupby('original_publication_year')['shared_n_text_reviews'].transform('max') == gb_max_reviews['shared_n_text_reviews']]
author_max_year

,original_publication_year,authors,shared_n_text_reviews
0,-720.0,Bernard Knox,1620.2
1,-720.0,E.V. Rieu,1620.2
2,-720.0,FrÃ©dÃ©ric Mugler,1620.2
3,-720.0,Homer,1620.2
4,-720.0,Robert Fagles,1620.2
...,...,...,...
129,2009.0,Suzanne Collins,88538.0
130,2010.0,Suzanne Collins,96274.0
132,2011.0,Veronica Roth,101023.0
135,2012.0,John Green,140739.0


# 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets ( riguardo i good_books, fare riferimento al dataset good_books_ratings. Sarebbe corretto, invece, utilizzare la colonna average_rating del dataset good_books?)


trovare i libri in entrambi i dataset: goodbook(average_rating) e books(average rating) key: ISBN
Differenza rating medio

In [28]:
rating_diff = goodbooks.merge(books_ratings_average, left_on='isbn', right_on='ISBN')[['ISBN','average_rating','Book-Rating_average']]
rating_diff['average_rating_diff'] = rating_diff['average_rating'] - rating_diff['Book-Rating_average']
rating_diff

,ISBN,average_rating,Book-Rating_average,average_rating_diff
0,1594480001,4.26,2.4,1.86
1,043965548X,4.53,3.5,1.03
2,1400032717,3.85,4.8,-0.95
3,014028009X,3.75,3.8,-0.05
4,014038572X,4.06,4.5,-0.44


In [29]:
rating_diff['average_rating_diff'] = rating_diff['average_rating_diff'].abs()
rating_diff

,ISBN,average_rating,Book-Rating_average,average_rating_diff
0,1594480001,4.26,2.4,1.86
1,043965548X,4.53,3.5,1.03
2,1400032717,3.85,4.8,0.95
3,014028009X,3.75,3.8,0.05
4,014038572X,4.06,4.5,0.44


# 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.



Problema con età troppo grandi

In [30]:
users['Age'].describe()

count    168096.000000
mean         34.751434
std          14.428097
min           0.000000
25%          24.000000
50%          32.000000
75%          44.000000
max         244.000000
Name: Age, dtype: float64

In [31]:
users['Age'] = users['Age'].fillna(-1)
users

,User-ID,Age,City,Region,Country
0,1,-1.0,nyc,new york,usa
1,2,18.0,stockton,california,usa
2,3,-1.0,moscow,yukon territory,russia
3,4,17.0,porto,v.n.gaia,portugal
4,5,-1.0,farnborough,hants,united kingdom
...,...,...,...,...,...
278853,278854,-1.0,portland,oregon,usa
278854,278855,50.0,tacoma,washington,united kingdom
278855,278856,-1.0,brampton,ontario,canada
278856,278857,-1.0,knoxville,tennessee,usa


In [32]:
bins= [-1,0,15,25,35,45,55,65,75,85,95,105,244]  ##Rimuovere le classi di età 105+ ?
labels = ['unknown_age','age_0_14','age_15_24','age_25_34','age_35_44','age_45_54','age_55_64','age_65_74','age_75_84','age_85_94','age_95_104','age_105+']
users['Age_Group'] = pd.cut(users['Age'], bins=bins, labels=labels, right=False)
users

,User-ID,Age,City,Region,Country,Age_Group
0,1,-1.0,nyc,new york,usa,unknown_age
1,2,18.0,stockton,california,usa,age_15_24
2,3,-1.0,moscow,yukon territory,russia,unknown_age
3,4,17.0,porto,v.n.gaia,portugal,age_15_24
4,5,-1.0,farnborough,hants,united kingdom,unknown_age
...,...,...,...,...,...,...
278853,278854,-1.0,portland,oregon,usa,unknown_age
278854,278855,50.0,tacoma,washington,united kingdom,age_45_54
278855,278856,-1.0,brampton,ontario,canada,unknown_age
278856,278857,-1.0,knoxville,tennessee,usa,unknown_age


In [35]:
users.groupby('Age_Group').count()

,User-ID,Age,City,Region,Country
Age_Group,,,,,
unknown_age,110762,110762,110762,110762,110762
age_0_14,4313,4313,4313,4313,4313
age_15_24,40001,40001,40001,40001,40001
age_25_34,50767,50767,50767,50766,50766
age_35_44,32690,32690,32690,32690,32690
age_45_54,23152,23152,23152,23152,23152
age_55_64,12493,12493,12493,12493,12493
age_65_74,3596,3596,3596,3596,3595
age_75_84,615,615,615,615,615


In [ ]:
#bins= [-1,0,15,25,35,45,55,65,75,85,95,105,115,125,135,145,155,165,175,185,195,205,215,225,235,245]  ##Rimuovere le classi di età 105+ ?
#labels = ['unknown_age','age_0_14','age_15_24','age_25_34','age_35_44','age_45_54','age_55_64','age_65_74','age_75_84','age_85_94','age_95_104','age_105_114','age_115_124','age_125_134',
#          'age_135_144','age_145_154','age_155_164','age_165_174','age_175_184','age_185_194','age_195_204','age_205_214','age_215_224','age_225_234','age_235_244']
#users['Age_grp'] = pd.cut(users['Age'], bins=bins, labels=labels, right=False)
#users

# 11. Find the books that appear only in the goodbooks datasets. (considerare ISBN)


In [38]:
only_goodbooks = goodbooks.loc[~goodbooks['isbn'].isin(books['ISBN'])].copy()
only_goodbooks

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,authors_count
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,[Suzanne Collins],2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,1
1,2,3,3,4640799,491,439554934,9.780440e+12,"[J.K. Rowling, Mary GrandPrÃ©]",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,2
2,3,41865,41865,3212258,226,316015849,9.780316e+12,[Stephenie Meyer],2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,1
3,4,2657,2657,3275794,487,61120081,9.780061e+12,[Harper Lee],1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,1
4,5,4671,4671,245494,1356,743273567,9.780743e+12,[F. Scott Fitzgerald],1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,5297,5297,1858012,2303,375751513,9.780376e+12,"[Oscar Wilde, Jeffrey Eugenides]",1891.0,The Picture of Dorian Gray,The Picture of Dorian Gray,eng,4.06,590014,674551,19647,9825,30735,129420,246892,257679,https://images.gr-assets.com/books/1424596966m...,https://images.gr-assets.com/books/1424596966s...,2
95,96,13536860,13536860,18034963,133,345803507,9.780346e+12,[E.L. James],2012.0,Fifty Shades Freed,"Fifty Shades Freed (Fifty Shades, #3)",eng,3.88,387290,610116,25287,39079,55691,109963,140304,265079,https://images.gr-assets.com/books/1336418837m...,https://images.gr-assets.com/books/1336418837s...,1
96,97,17245,17245,3165724,2207,393970124,9.780394e+12,"[Bram Stoker, Nina Auerbach, David J. Skal]",1897.0,Dracula,Dracula,eng,3.98,618973,689739,17263,11756,36719,151396,246043,243825,https://images.gr-assets.com/books/1387151694m...,https://images.gr-assets.com/books/1387151694s...,3
97,98,5060378,5060378,6976108,209,307269981,9.780307e+12,"[Stieg Larsson, Reg Keeland]",2006.0,Flickan som lekte med elden,"The Girl Who Played with Fire (Millennium, #2)",eng,4.22,563994,649253,31423,4623,15781,91021,258236,279592,https://images.gr-assets.com/books/1351778881m...,https://images.gr-assets.com/books/1351778881s...,2


# 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [39]:
n_author_title=books.groupby(['Book-Author','Book-Title'], as_index=False).size()
n_author_title

,Book-Author,Book-Title,size
0,D. Chiel,Die Hard: With a Vengeance - A Novel,1
1,J. D. Landis,The Taking,1
2,Mimma Balia,Ruskin's Rose: A Venetian Love Story,1
3,'N Sync,Nsync : The Official Book,1
4,142 moms from all over the world,The Mom-to-Mom Guide to the Baby Years: Review...,1
...,...,...,...
251178,Ã?dÃÂ¶n von HorvÃÂ¡th,Geschichten aus dem Wiener Wald (Gesammelte We...,1
251179,Ã?dÃÂ¶n von HorvÃÂ¡th,"SportmÃÂ¤rchen, andere Prosa und Verse (Gesam...",1
251180,Ã?Ã?dÃ?ÃÂ¶n von Horvath,"Suhrkamp BasisBibliothek (SBB), Nr.7, Jugend o...",1
251181,Ã?Ã?pictÃ?ÃÂ¨te,Maximes et pensÃ?ÃÂ©es : Ã?Ã?pictÃ?ÃÂ¨te,1


In [40]:
 n_author_title.loc[n_author_title['size'].idxmax()]

Book-Author    Louisa May Alcott
Book-Title          Little Women
size                          21
Name: 150649, dtype: object

Effettuare controllo  - una volta c'è errore di ortografia e un caso ha solo il cognome o il secondo nome è abbreviato

In [42]:
n_author_title[(n_author_title== 'Little Women').any(axis=1)]

,Book-Author,Book-Title,size
4239,Alcott,Little Women,1
142918,Laurie Lawlor,Little Women,1
150612,Louisa M. Atcott,Little Women,1
150649,Louisa May Alcott,Little Women,21


Per completezza visualizzare la classifica con gli altri libri in ordine decrescente per numerosità, in modo da valutare se la classifica si potrebbe teoricamente rivalutare 

In [43]:
n_author_title.sort_values(by=['size'], inplace=True, ascending=False)
n_author_title

,Book-Author,Book-Title,size
150649,Louisa May Alcott,Little Women,21
69474,Emily Bronte,Wuthering Heights,20
161302,Mark Twain,Adventures of Huckleberry Finn,20
106550,Jane Austen,Pride and Prejudice,18
75521,Frances Hodgson Burnett,The Secret Garden,16
...,...,...,...
86295,Grace Lebow,Coping With Your Difficult Older Parent : A Gu...,1
86296,Grace Lee Whitney,The Longest Trek: My Tour of the Galaxy,1
86297,Grace Livingst Hill,Finding of Jasper Holt,1
86298,Grace Livingst Hill,Strange Proposal Voice In the Wilderness,1


In [44]:
n_author_title[(n_author_title== 'Wuthering Heights').any(axis=1)]

,Book-Author,Book-Title,size
69474,Emily Bronte,Wuthering Heights,20
62573,EMILY BRONTE,Wuthering Heights,1


In [45]:
	n_author_title[(n_author_title== 'Adventures of Huckleberry Finn').any(axis=1)]

,Book-Author,Book-Title,size
161302,Mark Twain,Adventures of Huckleberry Finn,20


In [46]:
n_author_title[(n_author_title== 'Pride and Prejudice').any(axis=1)]

,Book-Author,Book-Title,size
106550,Jane Austen,Pride and Prejudice,18


In conclusione, si può affermare che la classifica per i primi libri nella graduatoria rimane tale anche senza considerare, ad esempio, variazioni nel nome dell'autore

# 13. Find the author with the highest average rating according to the goodbooks datasets.
 (per ogni autore bisogna calcolare un unico average rating complessivo, e restituire l'autore con media più elevata) 

i valori mancanti all'interno del dataset si possono lasciare all'interno dei risultati

In [ ]:
#authors_avg = goodbooks_new.groupby(['author'], as_index=False).mean()[['author','average_rating']]
#authors_avg.sort_values(by=['average_rating'], inplace=True, ascending=False)
#authors_avg.head()

In [49]:
tot_avg_rating_author=gb_authors.groupby(['authors'],as_index=False)['average_rating'].mean()
tot_avg_rating_author

,authors,average_rating
0,Alan R. Clarke,3.820
1,Aldous Huxley,3.970
2,Alice Sebold,3.770
3,Anne Frank,4.100
4,Antoine de Saint-ExupÃ©ry,4.280
...,...,...
104,Veronica Roth,4.155
105,William Golding,3.640
106,William Goldman,4.250
107,William Shakespeare,3.730


Estrapolare il nume dell'autore con "average_rating" massimo

In [56]:
tot_avg_rating_author.iloc[tot_avg_rating_author['average_rating'].idxmax()] 

authors           Rufus Beck
average_rating          4.53
Name: 91, dtype: object

Stampare la classifica in ordine decrescente per "average_rating"

In [ ]:
max_avg_rating=tot_avg_rating_author.sort_values('average_rating', ascending=False)
max_avg_rating

,authors,average_rating
91,Rufus Beck,4.530000
67,Mary GrandPrÃ©,4.497143
43,J.K. Rowling,4.497143
55,Kathryn Stockett,4.450000
34,George R.R. Martin,4.450000
...,...,...
41,Homer,3.730000
58,Lauren Weisberger,3.700000
98,Stephenie Meyer,3.664000
105,William Golding,3.640000
